In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import requests_cache
import time
import math
from sqlalchemy import create_engine
import execjs
import random
import io
import json

engine = create_engine('sqlite:///stock.sqlite')
conn = engine.connect()

exchange_types = {
    '上海证券交易所': '0',
    '深圳证券交易所': '1'
}

def get_netease_historical_data(exchange, code, start='19900101', end='20201126', fields='TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP'):
    url = 'http://quotes.money.163.com/service/chddata.html'
    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) ' 'Chrome/56.0.2924.87 Safari/537.36',
        'Referer': 'http://www.sse.com.cn/assortment/stock/list/share/'
    }
    data = {
        'code': exchange + str(code),
        'start': start,
        'end': end,
        'fields': fields,
    }
    r = requests.post(url, data=data, headers=headers)
    r.encoding='gb2312'
    df = pd.read_csv(io.StringIO(r.text))
    df.to_sql('historical_prices', engine, chunksize=1000, if_exists='append')
    print(str(code), '完成')

def get_netease_zycwzb(code):
    url = 'http://quotes.money.163.com/service/zycwzb_' + str(code) + '.html'
    data = {
        'type': 'report'
    }
    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) ' 'Chrome/56.0.2924.87 Safari/537.36',
        'Referer': 'http://www.sse.com.cn/assortment/stock/list/share/'
    }
    r = requests.post(url, data=data, headers=headers)
    return pd.read_csv(io.StringIO(r.text), index_col=0)

def update_historical_data_from_netease(end=None):
    if(end == None):
        end = datetime.now().strftime('%Y%m%d')
    stocks = pd.read_sql_table('stock_list', conn, index_col='index')
    stocks.apply(lambda row: get_netease_historical_data(exchange_types[row['交易所']], row['代码'], end=end), axis=1)

def regular(df):
    df.sort_index(inplace=True)
    df['股票代码'] = df['股票代码'].apply(lambda x: x.lstrip("'"))
    df['收盘价'].replace(0, np.NaN, inplace=True)
    df['收盘价'].fillna(method='ffill' , inplace=True)
    return df

In [24]:
code = '600036'

sql = "select * from historical_prices where 股票代码='''" + code + "'"
%time df = pd.read_sql_query(sql, conn, index_col='日期', parse_dates='日期')
# %time df.drop_duplicates(inplace=True)
df = regular(df)

cn_dividend_fields = {
    'SECURITY_CODE_A': 'A股股票代码',
    'RECORD_DATE_A': 'A股股权登记日',
    'EX_DIVIDEND_DATE_A': '除息日',
    'DIVIDEND_PER_SHARE1_A': '税后每股红利',
    'DIVIDEND_PER_SHARE2_A': '税前每股红利',
    'EXCHANGE_RATE': '-',
    'COMPANY_CODE': '公司代码',
    'FULL_NAME': '公司名称',
    'DIVIDEND_DATE': '股息日',
    'SECURITY_ABBR_A': '股票简称'
}

cn_bonu_fields = {
    'ANNOUNCE_DATE': '公告刊登日',
    'ANNOUNCE_DESTINATION': '公告宣布地',
    'BONUS_RATE': '送股比例',
    'CHANGE_RATE': '变动比例',
    'COMPANY_CODE': '股票代码',
    'COMPANY_NAME': '公司名称',
    'EX_RIGHT_DATE_A': '除权基准日',
    'EX_RIGHT_DATE_B': '除权基准日B',
    'LAST_TRADE_DATE_B': '最后交易日B',
    'RECORD_DATE_A': 'A股股权登记日',
    'RECORD_DATE_B': 'B股股权登记日',
    'SECURITY_CODE_A': 'A股证券代码',
    'SECURITY_CODE_B': 'B股证券代码',
    'SECURITY_NAME_A': 'A股证券名称',
    'SECURITY_NAME_B': 'B股证券名称',
    'TRADE_DATE_A': '红股上市日',
    'TRADE_DATE_B': 'B股红股上市日',
}

cn_allotments_fields = {
    'COMPANY_CODE': '公司代码',
    'END_DATE_OF_REMITTANCE_A': '配股缴款截止日',
    'EX_RIGHTS_DATE_A': 'A股除权交易日',
    'LISTING_DATE_A': '配股上市日',
    'PRICE_OF_RIGHTS_ISSUE_A': 'A股配股价格',
    'RATIO_OF_RIGHTS_ISSUE_A': '配股比例(10：?)',
    'RECORD_DATE_A': 'A股股权登记日',
    'SECURITY_CODE_A': 'A股证券代码',
    'SECURITY_NAME_A': '证券名称',
    'START_DATE_OF_REMITTANCE_A': '配股缴款起始日',
    'TRUE_COLUME_A': '实际配股量(万股)',
}

sql = 'select RECORD_DATE_A,DIVIDEND_PER_SHARE2_A from dividends where SECURITY_CODE_A="' + code + '"'
%time dividend = pd.read_sql_query(sql, conn, index_col='RECORD_DATE_A', parse_dates='RECORD_DATE_A')
%time dividend.rename(columns=cn_dividend_fields, inplace=True)

sql = 'select RECORD_DATE_A,BONUS_RATE from bonus where SECURITY_CODE_A="' + code + '"'
%time bonus = pd.read_sql_query(sql, conn, index_col='RECORD_DATE_A', parse_dates='RECORD_DATE_A')
%time bonus.rename(columns=cn_bonu_fields, inplace=True)

allotment_query_sql = 'select RECORD_DATE_A,EX_RIGHTS_DATE_A,PRICE_OF_RIGHTS_ISSUE_A,RATIO_OF_RIGHTS_ISSUE_A from allotments where SECURITY_CODE_A="' + code + '"'
%time allotments = pd.read_sql_query(allotment_query_sql, conn, index_col='RECORD_DATE_A', parse_dates='RECORD_DATE_A')
%time allotments.rename(columns=cn_allotments_fields, inplace=True)

CPU times: user 58.5 ms, sys: 4.19 ms, total: 62.6 ms
Wall time: 61.1 ms
CPU times: user 7.78 ms, sys: 0 ns, total: 7.78 ms
Wall time: 7.79 ms
CPU times: user 606 µs, sys: 32 µs, total: 638 µs
Wall time: 648 µs
CPU times: user 4.77 ms, sys: 0 ns, total: 4.77 ms
Wall time: 4.45 ms
CPU times: user 983 µs, sys: 30 µs, total: 1.01 ms
Wall time: 1.02 ms
CPU times: user 4.67 ms, sys: 0 ns, total: 4.67 ms
Wall time: 4.46 ms
CPU times: user 560 µs, sys: 0 ns, total: 560 µs
Wall time: 567 µs


In [70]:
%%time
dr = dividend[['税前每股红利']].astype(float).join(bonus['送股比例'].astype(float), how='outer')
dr = dr.join(allotments[['A股配股价格', '配股比例(10：?)']].astype(float), how='outer')
dr = dr.join(df['收盘价'], how='left')
dr['送股比例'] /= 10
dr['配股比例(10：?)'] /= 10
dr.fillna(0, inplace=True)
ex_dividend_and_right = lambda x: (x['收盘价'] - x['税前每股红利'] + x['A股配股价格'] * x['配股比例(10：?)']) / (1 + x['送股比例'] + x['配股比例(10：?)'])
df['除息除权价'] = dr[(dr['税前每股红利'] > 0) | (dr['送股比例'] > 0) | dr['配股比例(10：?)'] > 0].apply(ex_dividend_and_right, axis=1)
df['除息除权价'].round(2)
df['收益率'] = df['收盘价'].pct_change()
df['复权收益率'] = (df['收盘价'] / df['除息除权价'].shift(1).round(2)) - 1
df['复权收益率'].mask(df['复权收益率'].isna(), df['收盘价'].pct_change(), inplace=True)
df['复权价'] = df.iloc[0]['收盘价'] * (1 + df['复权收益率']).cumprod()

CPU times: user 24.1 ms, sys: 0 ns, total: 24.1 ms
Wall time: 23.7 ms


In [71]:
df['自诉收益率'] = df['收盘价'] / df['前收盘'] - 1
df.loc['2002-4-9', '自诉收益率'] = np.NaN
df['复权价-2'] = df.iloc[0]['收盘价'] * (1 + df['自诉收益率']).cumprod()

In [74]:
df

,index,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,...,成交量,成交金额,总市值,流通市值,除息除权价,收益率,复权收益率,复权价,自诉收益率,复权价-2
日期,,,,,,,,,,,,,,,,,,,,,
2002-04-09,4537,600036,N 招商银,10.66,10.88,10.51,10.51,7.30,3.36,46.0274,...,414108831,4.418822e+09,6.083468e+10,6.396000e+09,NaN,NaN,NaN,NaN,NaN,NaN
2002-04-10,4536,600036,招商银行,10.60,10.70,10.39,10.66,10.66,-0.06,-0.5629,...,67945475,7.166843e+08,6.049227e+10,6.360000e+09,NaN,-0.005629,-0.005629,10.600000,-0.005629,10.600000
2002-04-11,4535,600036,招商银行,10.52,10.68,10.49,10.60,10.60,-0.08,-0.7547,...,22788255,2.409635e+08,6.003573e+10,6.312000e+09,NaN,-0.007547,-0.007547,10.520000,-0.007547,10.520000
2002-04-12,4534,600036,招商银行,10.57,10.64,10.48,10.50,10.52,0.05,0.4753,...,21256463,2.240599e+08,6.032107e+10,6.342000e+09,NaN,0.004753,0.004753,10.570000,0.004753,10.570000
2002-04-15,4533,600036,招商银行,10.39,10.60,10.35,10.57,10.57,-0.18,-1.7029,...,18531104,1.933069e+08,5.929384e+10,6.234000e+09,NaN,-0.017029,-0.017029,10.390000,-0.017029,10.390000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-04,4,600036,招商银行,45.78,46.60,45.11,46.47,46.49,-0.71,-1.5272,...,46115543,2.099378e+09,1.154565e+12,9.443931e+11,NaN,-0.015272,-0.015272,199.711039,-0.015272,216.942118
2020-12-07,3,600036,招商银行,44.42,45.75,44.10,45.70,45.78,-1.36,-2.9707,...,70482524,3.138039e+09,1.120266e+12,9.163377e+11,NaN,-0.029707,-0.029707,193.778164,-0.029707,210.497355
2020-12-08,2,600036,招商银行,44.26,44.76,44.00,44.39,44.42,-0.16,-0.3602,...,37716735,1.671238e+09,1.116230e+12,9.130371e+11,NaN,-0.003602,-0.003602,193.080178,-0.003602,209.739147


In [69]:
df['除息除权价'].shift(1).round(2).loc['2006-2-24']

7.07

In [73]:
print(df.loc['2006-2-23'])
print(df.loc['2006-2-24'])

index           3602
股票代码          600036
名称              招商银行
收盘价             7.68
最高价                0
最低价                0
开盘价                0
前收盘             7.68
涨跌额             None
涨跌幅             None
换手率                0
成交量                0
成交金额               0
总市值      8.68442e+10
流通市值     2.86891e+10
除息除权价        7.07254
收益率                0
复权收益率              0
复权价          14.2563
自诉收益率              0
复权价-2        14.2563
Name: 2006-02-23 00:00:00, dtype: object
index           3601
股票代码          600036
名称             XR招商银
收盘价             7.68
最高价                0
最低价                0
开盘价                0
前收盘             7.07
涨跌额             None
涨跌幅             None
换手率                0
成交量                0
成交金额               0
总市值      7.99464e+10
流通市值     2.64104e+10
除息除权价            NaN
收益率                0
复权收益率      0.0862801
复权价          15.4863
自诉收益率      0.0862801
复权价-2        15.4863
Name: 2006-02-24 00:00:00, dtype: object


In [39]:
df.loc['2013-9-5']

index           1768
股票代码          600036
名称             XR招商银
收盘价            10.66
最高价            10.75
最低价            10.61
开盘价             10.7
前收盘            10.46
涨跌额              0.2
涨跌幅            1.912
换手率           0.5715
成交量        117901460
成交金额      1.2609e+09
总市值       2.6159e+11
流通市值     2.19905e+11
除息除权价            NaN
收益率                0
复权收益率              0
复权价          35.8138
Name: 2013-09-05 00:00:00, dtype: object

In [26]:
df

,index,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值,除息除权价,收益率,复权收益率,复权价
日期,,,,,,,,,,,,,,,,,,,
2002-04-09,4537,600036,N 招商银,10.66,10.88,10.51,10.51,7.30,3.36,46.0274,69.0181,414108831,4.418822e+09,6.083468e+10,6.396000e+09,NaN,NaN,NaN,NaN
2002-04-10,4536,600036,招商银行,10.60,10.70,10.39,10.66,10.66,-0.06,-0.5629,11.3242,67945475,7.166843e+08,6.049227e+10,6.360000e+09,NaN,-0.005629,-0.005629,10.600000
2002-04-11,4535,600036,招商银行,10.52,10.68,10.49,10.60,10.60,-0.08,-0.7547,3.7980,22788255,2.409635e+08,6.003573e+10,6.312000e+09,NaN,-0.007547,-0.007547,10.520000
2002-04-12,4534,600036,招商银行,10.57,10.64,10.48,10.50,10.52,0.05,0.4753,3.5427,21256463,2.240599e+08,6.032107e+10,6.342000e+09,NaN,0.004753,0.004753,10.570000
2002-04-15,4533,600036,招商银行,10.39,10.60,10.35,10.57,10.57,-0.18,-1.7029,3.0885,18531104,1.933069e+08,5.929384e+10,6.234000e+09,NaN,-0.017029,-0.017029,10.390000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-04,4,600036,招商银行,45.78,46.60,45.11,46.47,46.49,-0.71,-1.5272,0.2235,46115543,2.099378e+09,1.154565e+12,9.443931e+11,NaN,-0.015272,-0.015272,199.726136
2020-12-07,3,600036,招商银行,44.42,45.75,44.10,45.70,45.78,-1.36,-2.9707,0.3417,70482524,3.138039e+09,1.120266e+12,9.163377e+11,NaN,-0.029707,-0.029707,193.792813
2020-12-08,2,600036,招商银行,44.26,44.76,44.00,44.39,44.42,-0.16,-0.3602,0.1828,37716735,1.671238e+09,1.116230e+12,9.130371e+11,NaN,-0.003602,-0.003602,193.094775


In [28]:
dr['前收盘价'] = df['前收盘'].shift(1)
dr.join(df['前收盘'].shift(1), how='left')

,税前每股红利,送股比例,A股配股价格,配股比例(10：?),收盘价,前收盘价,前收盘
RECORD_DATE_A,,,,,,,
2003-07-15,0.120,0.00000,0.00,0.000,12.04,12.01,12.01
2004-05-10,0.092,0.20000,0.00,0.000,10.60,10.83,10.83
2005-06-17,0.110,0.50000,0.00,0.000,8.90,8.58,8.58
2006-02-23,0.000,0.08589,0.00,0.000,7.68,7.68,7.68
2006-06-15,0.080,0.00000,0.00,0.000,7.02,6.99,6.99
2006-09-20,0.180,0.00000,0.00,0.000,9.54,9.19,9.19
2007-07-03,0.120,0.00000,0.00,0.000,24.42,24.58,24.58
2008-07-25,0.280,0.00000,0.00,0.000,25.00,24.07,24.07
2009-07-02,0.100,0.30000,0.00,0.000,22.98,22.41,22.41


In [29]:
dr['除权价'] = df['除息除权价']
dr

,税前每股红利,送股比例,A股配股价格,配股比例(10：?),收盘价,前收盘价,除权价
RECORD_DATE_A,,,,,,,
2003-07-15,0.120,0.00000,0.00,0.000,12.04,12.01,11.920000
2004-05-10,0.092,0.20000,0.00,0.000,10.60,10.83,8.756667
2005-06-17,0.110,0.50000,0.00,0.000,8.90,8.58,5.860000
2006-02-23,0.000,0.08589,0.00,0.000,7.68,7.68,7.072540
2006-06-15,0.080,0.00000,0.00,0.000,7.02,6.99,6.940000
2006-09-20,0.180,0.00000,0.00,0.000,9.54,9.19,9.360000
2007-07-03,0.120,0.00000,0.00,0.000,24.42,24.58,24.300000
2008-07-25,0.280,0.00000,0.00,0.000,25.00,24.07,24.720000
2009-07-02,0.100,0.30000,0.00,0.000,22.98,22.41,17.600000


In [31]:
df.loc['2020-7-10']

index            103
股票代码          600036
名称             XD招商银
收盘价            37.25
最高价               39
最低价            37.05
开盘价            38.71
前收盘            38.71
涨跌额            -1.46
涨跌幅          -3.7716
换手率           0.7408
成交量        152815945
成交金额     5.78478e+09
总市值      9.39439e+11
流通市值     7.68428e+11
除息除权价            NaN
收益率         -0.06665
复权收益率     -0.0377164
复权价          162.512
Name: 2020-07-10 00:00:00, dtype: object